In [1]:
from KorAPClient import KorAPConnection
from rpy2.robjects import r

# As base, use the fiction corpus DeLiKo@DNB (see <https://doi.org/10.5281/zenodo.14943116>)
kcon = KorAPConnection(KorAPUrl="https://korap.dnb.de/", verbose=True).auth()

r['set.seed'](42) # Set the seed for reproducibility, will in future be exported by KorAPClient
q = kcon.corpusQuery("[tt/l=Korpus]", metadataOnly=False)
q = q.fetchNext(maxFetch=1000, randomizePageOrder=True)

R[write to console]: Welcome to KorAP API for DeLiKo@DNB!



Searching "[tt/l=Korpus]" in "" by this KorAP instance.: 771 hits, took 0.11682251 s
Using fields API: Retrieved page 1/16 in 0.31968938 s
Using fields API: Retrieved page 2/16 in 0.384494092 s
Using fields API: Retrieved page 3/16 in 0.229332277 s
Using fields API: Retrieved page 3/16 in 0.229332277 s
Using fields API: Retrieved page 4/16 in 0.229332277 s
Using fields API: Retrieved page 5/16 in 0.397982977 s
Using fields API: Retrieved page 6/16 in 0.467999036 s
Using fields API: Retrieved page 7/16 in 1.06042362 s
Using fields API: Retrieved page 8/16 in 0.504681485 s
Using fields API: Retrieved page 9/16 in 0.436383899 s
Using fields API: Retrieved page 10/16 in 0.621623505 s
Using fields API: Retrieved page 11/16 in 0.258548927 s
Using fields API: Retrieved page 12/16 in 0.433153669 s
Using fields API: Retrieved page 13/16 in 0.432334822 s
Using fields API: Retrieved page 14/16 in 0.399952292 s
Using fields API: Retrieved page 15/16 in 0.400414105 s
Using fields API: Retrieved pag

In [ ]:
results = q.slots['collectedMatches']
# use index as line_id
results["line_id"] = results.index

# take all columns except snippet, tokens.left, tokens.match, tokens.right, matchStart, matchEnd
drop_columns = ['snippet', 'tokens.left', 'tokens.match', 'tokens.right', 'matchStart', 'matchEnd']
metadata = results.drop(columns=drop_columns)

tokens = []
for _, line in results.iterrows():
    left_context, match, right_context = (line[col].split("\t") for col in ['tokens.left', 'tokens.match', 'tokens.right'])
    id_in_line = -1 # will be obsolete in later version of flexiconc
    
    if left_context != [""]:
        for i, token in enumerate(left_context):
            id_in_line += 1
            offset = 0 - len(left_context) + i
            tokens.append([offset, token, line["line_id"], id_in_line])
    
    for i, token in enumerate(match):
        id_in_line += 1
        offset = 0
        tokens.append([offset, token, line["line_id"], id_in_line])
    
    if right_context != [""]:
        for i, token in enumerate(right_context):
            id_in_line += 1
            offset = i + 1
            tokens.append([offset, token, line["line_id"], id_in_line])

# create a dataframe from the tokens list
import pandas as pd
tokens = pd.DataFrame(tokens, columns=["offset", "word", "line_id", "id_in_line"])

# matches df will be obsolete in later version of flexiconc as "offset" is included in tokens
# Create the matches DataFrame using the index directly for aggregation
matches = tokens[tokens['offset'] == 0].groupby('line_id').apply(
    lambda group: pd.Series({
        'match_start': group.index.min(),  # Get the minimum index value for match_start
        'match_end': group.index.max()  # Get the maximum index value for match_end
    })
).reset_index()

# Add 'slot' column to the matches DataFrame and populate it with 0's
matches['slot'] = 0

/var/folders/76/__9t5rnd5k94skg1118jhpw00000gn/T/ipykernel_77388/1758551592.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches = tokens[tokens['offset'] == 0].groupby('line_id').apply(


In [ ]:
from flexiconc.concordance import Concordance

# Create the Concordance object
c = Concordance()

# Option A: Tokens file includes 'offset'
c.load(
    metadata=metadata,
    tokens=tokens,
    matches=matches
)


In [9]:
from IPython.display import HTML, display
from flexiconc.visualization.html_visualizer import generate_concordance_html, generate_analysis_tree_html
display(HTML(generate_analysis_tree_html(c)))

In [10]:
display(HTML(generate_concordance_html(c, c.root, n=20)))

Line ID,Left Context,Node,Right Context
0,"des Attila, der seinen stattlichen",Korpus,viel zu eng umspannt hielt
1,– einem mannshohen Streichinstrument mit gekrümmtem,Korpus,"in Form eines Halbmonds,"
2,an ihren Trommelfellen und brachte den,Korpus,des Wagens zum Wackeln.
3,an den Schnittstellen zwischen Tür und,Korpus,". Genau da, wo"
4,ein halbes Dutzend Klingen aus dem,Korpus,schnellte. Und dann kam
5,gebürstete Oberfläche seiner Gliedmaßen verlieh dem,Korpus,"etwas Edles, während die"
6,"Multitool, ein Reparaturbot mit flachem",Korpus,", der seine sechs flexiblen"
7,"gehabt, die Kennung an seinem",Korpus,"auszumachen, würde aber schnellstmöglich"
8,"Art von Reparaturbots, mit flachem",Korpus,"und mehreren Gliedmaßen, die"
9,"und mit gesplittertem Visor, der",Korpus,"eines Roboters, dessen Arme"
